In [18]:
import pandas as pd
import yfinance as yf

from sqlalchemy import create_engine
from datetime import timedelta

In [19]:
raw_data = yf.download(
    "BTC-USD", start=pd.to_datetime("today") - timedelta(7), interval="1m"
)

[*********************100%***********************]  1 of 1 completed


In [20]:
raw_data["pct_change"] = raw_data["Close"].pct_change()
raw_data.dropna(inplace=True)


In [21]:
raw_data.shape


(9260, 7)

In [22]:
try:
    engine = create_engine("sqlite:///price_database.db")
    raw_data.to_sql("BTC_price_1m", engine)
except:
    pass


In [23]:
max_date = pd.read_sql("SELECT MAX(Datetime) FROM BTC_price_1m", engine).values[0][0]


In [24]:
new_rows = raw_data[raw_data.index > max_date]


In [25]:
new_rows.to_sql("BTC_price_1m", engine, if_exists="append")
print(f"{len(new_rows)} new rows of data have been appended")

latest = pd.read_sql("SELECT MAX(Datetime) FROM BTC_price_1m", engine).values[0][0]
print(f"Latest data datetime: {latest} (-5h difference)")


0 new rows of data have been appended
Latest data datetime: 2023-04-11 08:38:00.000000 (-5h difference)


In [26]:
len(pd.read_sql("SELECT * FROM BTC_price_1m", engine))

9260